In [5]:
! pip install kafka-python

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3108, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2901, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 441, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 

In [1]:
! pip install confluent-kafka

In [3]:
from confluent_kafka import Producer
import json

def produce_inventory_order():
    # Kafka producer configuration
    kafka_config = {'bootstrap.servers': 'localhost:9092'}

    # Create Kafka producer
    producer = Producer(kafka_config)

    # Simulate inventory events data (replace this with actual data source)
    inventory_events = [
        {"type": "inventory", "item_id": "123", "quantity": 10},
        {"type": "inventory", "item_id": "456", "quantity": 20}
    ]

    # Send inventory events to Kafka topic
    for event in inventory_events:
        producer.produce('inventory_orders', json.dumps(event).encode('utf-8'))

    # Flush producer to send messages
    producer.flush()

def produce_delivery_order():
    # Kafka producer configuration
    kafka_config = {'bootstrap.servers': 'localhost:9092'}

    # Create Kafka producer
    producer = Producer(kafka_config)

    # Simulate delivery events data (replace this with actual data source)
    delivery_events = [
        {"type": "delivery", "order_id": "1001", "status": "pending"},
        {"type": "delivery", "order_id": "1002", "status": "shipped"}
    ]

    # Send delivery events to Kafka topic
    for event in delivery_events:
        producer.produce('delivery_orders', json.dumps(event).encode('utf-8'))

    # Flush producer to send messages
    producer.flush()

# Produce inventory and delivery orders
produce_inventory_order()
produce_delivery_order()

In [1]:
from confluent_kafka import Consumer, KafkaError

def consume_inventory_data():
    # Kafka consumer configuration
    kafka_config = {'bootstrap.servers': 'localhost:9092', 'group.id': 'inventory_group'}

    # Create Kafka consumer
    consumer = Consumer(kafka_config)
    consumer.subscribe(['inventory_orders'])

    # Consume messages
    while True:
        msg = consumer.poll(timeout=1.0)  # Poll for messages
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition
                continue
            else:
                print(f"Consumer error: {msg.error()}")
                break
        # Process inventory message
        inventory_data = json.loads(msg.value().decode('utf-8'))
        print("Received inventory data:", inventory_data)
        # Perform actions like updating inventory databases

def consume_delivery_data():
    # Kafka consumer configuration
    kafka_config = {'bootstrap.servers': 'localhost:9092', 'group.id': 'delivery_group'}

    # Create Kafka consumer
    consumer = Consumer(kafka_config)
    consumer.subscribe(['delivery_orders'])

    # Consume messages
    while True:
        msg = consumer.poll(timeout=1.0)  # Poll for messages
        if msg is None:
            continue
        if msg.error():
            if msg.error().code() == KafkaError._PARTITION_EOF:
                # End of partition
                continue
            else:
                print(f"Consumer error: {msg.error()}")
                break
        # Process delivery message
        delivery_data = json.loads(msg.value().decode('utf-8'))
        print("Received delivery data:", delivery_data)
        # Perform actions like scheduling deliveries, updating status, etc.

# Consume inventory and delivery data
consume_inventory_data()
consume_delivery_data()

In [ ]:
import json

def filter_inventory_message(message):
    try:
        data = json.loads(message)
        return data.get('type') == 'inventory'
    except json.JSONDecodeError:
        return False

def filter_delivery_message(message):
    try:
        data = json.loads(message)
        return data.get('type') == 'delivery'
    except json.JSONDecodeError:
        return False